# Joins y Agregaciones Avanzadas

## Objetivos de Aprendizaje
- Dominar todos los tipos de joins en Spark
- Aplicar agregaciones complejas con múltiples niveles
- Usar pivot y unpivot para reestructurar datos
- Optimizar joins para mejor rendimiento

## Prerequisitos
- `06_spark_processing/03_spark_sql.ipynb`

## Tiempo Estimado
⏱️ 60 minutos

## Módulo AWS Academy Relacionado
📚 Módulo 9: Big Data Processing - Operaciones de datos complejas

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from datetime import date

spark = SparkSession.builder \
    .appName("JoinsAggregations") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")
print("Spark listo")

In [ ]:
# Crear DataFrames de ejemplo mas completos
# Empleados
empleados = spark.createDataFrame([
    (1, "Ana", "Ventas", 45000, date(2020, 3, 15)),
    (2, "Carlos", "IT", 55000, date(2019, 7, 1)),
    (3, "Maria", "Ventas", 48000, date(2021, 1, 10)),
    (4, "Juan", "IT", 52000, date(2020, 6, 20)),
    (5, "Laura", "Marketing", 42000, date(2022, 2, 1)),
    (6, "Pedro", "IT", 60000, date(2018, 11, 5)),
    (7, "Sofia", "Ventas", 47000, date(2021, 8, 15)),
    (8, "Diego", None, 38000, date(2023, 1, 1))  # Sin departamento
], ["id", "nombre", "departamento", "salario", "fecha_ingreso"])

# Departamentos
departamentos = spark.createDataFrame([
    ("Ventas", "Piso 1", 100000),
    ("IT", "Piso 2", 200000),
    ("Marketing", "Piso 1", 80000),
    ("RRHH", "Piso 3", 60000)  # Sin empleados
], ["nombre_depto", "ubicacion", "presupuesto"])

# Proyectos
proyectos = spark.createDataFrame([
    (101, "Web App", "IT", date(2024, 1, 1), date(2024, 6, 30)),
    (102, "Campaña Q1", "Marketing", date(2024, 1, 1), date(2024, 3, 31)),
    (103, "CRM", "Ventas", date(2024, 2, 1), date(2024, 12, 31)),
    (104, "Mobile App", "IT", date(2024, 3, 1), date(2024, 9, 30))
], ["proyecto_id", "nombre_proyecto", "departamento", "inicio", "fin"])

# Asignaciones empleado-proyecto
asignaciones = spark.createDataFrame([
    (1, 103, 20),  # Ana en CRM, 20 horas/semana
    (2, 101, 40),  # Carlos en Web App
    (3, 103, 30),  # Maria en CRM
    (4, 101, 20),  # Juan en Web App
    (4, 104, 20),  # Juan tambien en Mobile
    (5, 102, 40),  # Laura en Campaña
    (6, 104, 40),  # Pedro en Mobile
    (7, 103, 25)   # Sofia en CRM
], ["empleado_id", "proyecto_id", "horas_semana"])

print("DataFrames creados")

---
# === SECCIÓN 1 ===
## 1. Tipos de Joins

### Explicación Conceptual
Un **JOIN** combina filas de dos tablas basándose en una condición (generalmente igualdad de columnas).

**Tipos:**
- `inner`: Solo coincidencias en ambas tablas
- `left` / `left_outer`: Todo de la izquierda + coincidencias
- `right` / `right_outer`: Todo de la derecha + coincidencias
- `full` / `full_outer`: Todo de ambas tablas
- `cross`: Producto cartesiano (cada fila con cada fila)
- `left_semi`: Filas de izquierda que tienen match (sin columnas de derecha)
- `left_anti`: Filas de izquierda que NO tienen match

In [ ]:
# INNER JOIN: Solo empleados con departamento valido
print("INNER JOIN - Empleados con departamento:")
empleados.join(
    departamentos,
    empleados["departamento"] == departamentos["nombre_depto"],
    "inner"
).select("nombre", "departamento", "ubicacion", "salario").show()

In [ ]:
# LEFT JOIN: Todos los empleados, con o sin departamento
print("LEFT JOIN - Todos los empleados:")
empleados.join(
    departamentos,
    empleados["departamento"] == departamentos["nombre_depto"],
    "left"
).select("nombre", "departamento", "ubicacion").show()

In [ ]:
# RIGHT JOIN: Todos los departamentos, con o sin empleados
print("RIGHT JOIN - Todos los departamentos:")
empleados.join(
    departamentos,
    empleados["departamento"] == departamentos["nombre_depto"],
    "right"
).select("nombre_depto", "nombre", "salario").show()

In [ ]:
# LEFT SEMI: Empleados que SI estan en algun proyecto
print("LEFT SEMI - Empleados asignados a proyectos:")
empleados.join(
    asignaciones,
    empleados["id"] == asignaciones["empleado_id"],
    "left_semi"  # Solo retorna columnas de la tabla izquierda
).show()

In [ ]:
# LEFT ANTI: Empleados que NO estan en ningun proyecto
print("LEFT ANTI - Empleados sin proyectos:")
empleados.join(
    asignaciones,
    empleados["id"] == asignaciones["empleado_id"],
    "left_anti"
).show()

---
# === SECCIÓN 2 ===
## 2. Joins Múltiples

### Explicación Conceptual
Frecuentemente necesitamos unir más de dos tablas para obtener una vista completa de los datos.

In [ ]:
# Join de 4 tablas: Vista completa de asignaciones
vista_completa = asignaciones \
    .join(empleados, asignaciones["empleado_id"] == empleados["id"]) \
    .join(proyectos, asignaciones["proyecto_id"] == proyectos["proyecto_id"]) \
    .join(
        departamentos, 
        empleados["departamento"] == departamentos["nombre_depto"],
        "left"
    ) \
    .select(
        empleados["nombre"].alias("empleado"),
        empleados["departamento"],
        proyectos["nombre_proyecto"].alias("proyecto"),
        asignaciones["horas_semana"],
        departamentos["ubicacion"]
    )

print("Vista completa de asignaciones:")
vista_completa.show(truncate=False)

---
# === SECCIÓN 3 ===
## 3. Agregaciones Avanzadas

### Explicación Conceptual
Las agregaciones resumen datos. Spark permite agregaciones múltiples, condicionales y anidadas.

In [ ]:
# Agregaciones multiples por grupo
print("Estadisticas por departamento:")
empleados.filter(F.col("departamento").isNotNull()) \
    .groupBy("departamento") \
    .agg(
        F.count("*").alias("num_empleados"),
        F.sum("salario").alias("salario_total"),
        F.round(F.avg("salario"), 2).alias("salario_promedio"),
        F.min("salario").alias("salario_min"),
        F.max("salario").alias("salario_max"),
        F.round(F.stddev("salario"), 2).alias("desviacion_std")
    ) \
    .orderBy(F.desc("num_empleados")) \
    .show()

In [ ]:
# Agregacion condicional con CASE/WHEN
print("Conteo condicional:")
empleados.agg(
    F.count("*").alias("total_empleados"),
    F.sum(F.when(F.col("salario") > 50000, 1).otherwise(0)).alias("salario_alto"),
    F.sum(F.when(F.col("salario") <= 50000, 1).otherwise(0)).alias("salario_normal"),
    F.sum(F.when(F.year(F.col("fecha_ingreso")) >= 2022, 1).otherwise(0)).alias("nuevos_2022+")
).show()

In [ ]:
# collect_list y collect_set: Agregar valores en listas
print("Empleados por departamento (lista):")
empleados.filter(F.col("departamento").isNotNull()) \
    .groupBy("departamento") \
    .agg(
        F.collect_list("nombre").alias("empleados"),
        F.collect_set("nombre").alias("empleados_unicos")  # Sin duplicados
    ).show(truncate=False)

---
# === SECCIÓN 4 ===
## 4. Pivot y Unpivot

### Explicación Conceptual
**Pivot** transforma filas en columnas (de formato largo a ancho).
**Unpivot** hace lo contrario (de ancho a largo).

**Analogía:** Es como reorganizar una tabla de Excel. Pivot es cuando conviertes categorías de una columna en múltiples columnas separadas.

In [ ]:
# Crear datos de ventas mensuales para pivot
ventas_mensuales = spark.createDataFrame([
    ("Norte", "Enero", 15000),
    ("Norte", "Febrero", 18000),
    ("Norte", "Marzo", 20000),
    ("Sur", "Enero", 12000),
    ("Sur", "Febrero", 14000),
    ("Sur", "Marzo", 16000),
    ("Centro", "Enero", 22000),
    ("Centro", "Febrero", 25000),
    ("Centro", "Marzo", 28000)
], ["region", "mes", "ventas"])

print("Datos originales (formato largo):")
ventas_mensuales.show()

In [ ]:
# PIVOT: Convertir meses a columnas
print("Despues de PIVOT (formato ancho):")
ventas_pivot = ventas_mensuales \
    .groupBy("region") \
    .pivot("mes", ["Enero", "Febrero", "Marzo"]) \
    .sum("ventas")

ventas_pivot.show()

In [ ]:
# UNPIVOT: Volver al formato largo
# Spark no tiene unpivot directo, se usa stack()
print("UNPIVOT (volver a formato largo):")
ventas_unpivot = ventas_pivot.select(
    "region",
    F.expr("stack(3, 'Enero', Enero, 'Febrero', Febrero, 'Marzo', Marzo) as (mes, ventas)")
)

ventas_unpivot.show()

---
# === SECCIÓN 5 ===
## 5. Agregaciones con Rollup y Cube

### Explicación Conceptual
- **ROLLUP**: Crea subtotales jerárquicos (de más detalle a menos)
- **CUBE**: Crea todas las combinaciones posibles de subtotales

In [ ]:
# Datos para demostrar rollup/cube
ventas_detalle = spark.createDataFrame([
    ("Norte", "Electronica", 2024, 150000),
    ("Norte", "Muebles", 2024, 80000),
    ("Sur", "Electronica", 2024, 120000),
    ("Sur", "Muebles", 2024, 60000),
    ("Norte", "Electronica", 2023, 140000),
    ("Norte", "Muebles", 2023, 75000),
    ("Sur", "Electronica", 2023, 110000),
    ("Sur", "Muebles", 2023, 55000)
], ["region", "categoria", "anio", "ventas"])

In [ ]:
# ROLLUP: Subtotales jerarquicos
print("ROLLUP - Subtotales por region -> categoria:")
ventas_detalle.rollup("region", "categoria") \
    .agg(F.sum("ventas").alias("total_ventas")) \
    .orderBy("region", "categoria") \
    .show()

# Las filas con NULL son subtotales
# NULL, NULL = gran total
# region, NULL = total por region

In [ ]:
# CUBE: Todas las combinaciones
print("CUBE - Todas las combinaciones de subtotales:")
ventas_detalle.cube("region", "categoria") \
    .agg(F.sum("ventas").alias("total_ventas")) \
    .orderBy("region", "categoria") \
    .show()

# Incluye totales por cada dimension individualmente

---
# === EJERCICIOS PRÁCTICOS ===

### 🎯 Ejercicio J.1: Análisis de Proyectos

Crea un reporte que muestre para cada proyecto:
- Nombre del proyecto
- Departamento responsable
- Número de empleados asignados
- Total de horas semanales
- Costo semanal (horas × salario/40)

In [ ]:
# TODO: Completa el ejercicio


### ✅ Solución Ejercicio J.1

In [ ]:
reporte_proyectos = proyectos \
    .join(asignaciones, "proyecto_id") \
    .join(empleados, asignaciones["empleado_id"] == empleados["id"]) \
    .groupBy(
        proyectos["proyecto_id"],
        proyectos["nombre_proyecto"],
        proyectos["departamento"]
    ) \
    .agg(
        F.count("*").alias("num_empleados"),
        F.sum("horas_semana").alias("total_horas"),
        F.round(
            F.sum(F.col("horas_semana") * F.col("salario") / 40 / 52),
            2
        ).alias("costo_semanal")
    ) \
    .orderBy(F.desc("costo_semanal"))

print("Reporte de proyectos:")
reporte_proyectos.show()

### 🎯 Ejercicio J.2: Matriz de Asignación

Crea una matriz (pivot) que muestre:
- Filas: Nombres de empleados
- Columnas: Nombres de proyectos
- Valores: Horas semanales (0 si no está asignado)

In [ ]:
# TODO: Completa el ejercicio


### ✅ Solución Ejercicio J.2

In [ ]:
# Obtener lista de proyectos para el pivot
lista_proyectos = [row.nombre_proyecto for row in proyectos.select("nombre_proyecto").collect()]

matriz = empleados \
    .join(asignaciones, empleados["id"] == asignaciones["empleado_id"], "left") \
    .join(proyectos, "proyecto_id", "left") \
    .groupBy(empleados["nombre"].alias("empleado")) \
    .pivot("nombre_proyecto", lista_proyectos) \
    .agg(F.coalesce(F.sum("horas_semana"), F.lit(0))) \
    .fillna(0)

print("Matriz de asignacion:")
matriz.show()

### 🎯 Ejercicio J.3: Reporte con Rollup

Usando los datos de empleados:
1. Crea un reporte con ROLLUP por departamento y año de ingreso
2. Muestra conteo y salario promedio
3. Añade una columna que indique si es subtotal

In [ ]:
# TODO: Completa el ejercicio


### ✅ Solución Ejercicio J.3

In [ ]:
reporte_rollup = empleados \
    .filter(F.col("departamento").isNotNull()) \
    .withColumn("anio_ingreso", F.year(F.col("fecha_ingreso"))) \
    .rollup("departamento", "anio_ingreso") \
    .agg(
        F.count("*").alias("num_empleados"),
        F.round(F.avg("salario"), 2).alias("salario_promedio")
    ) \
    .withColumn("tipo_fila",
        F.when(F.col("departamento").isNull(), "GRAN TOTAL")
        .when(F.col("anio_ingreso").isNull(), "SUBTOTAL DEPTO")
        .otherwise("DETALLE")
    ) \
    .orderBy("departamento", "anio_ingreso")

print("Reporte con ROLLUP:")
reporte_rollup.show()

---
# === RESUMEN FINAL ===

## Resumen

### Conceptos Clave
- **Joins**: `inner`, `left`, `right`, `full`, `semi`, `anti` para combinar datos
- **Agregaciones**: `groupBy` + `agg` con múltiples funciones
- **Pivot/Unpivot**: Reestructurar entre formato largo y ancho
- **Rollup/Cube**: Subtotales jerárquicos y multidimensionales
- **collect_list/set**: Agregar valores en arrays

### Conexión con AWS
- **Athena**: Soporta joins y agregaciones SQL similares
- **Redshift**: Data warehouse con estas operaciones optimizadas
- **Glue**: ETL con estas transformaciones

### Siguiente Paso
Continúa con: `05_window_functions.ipynb` para funciones de ventana avanzadas